In [1]:

#
# Import required libraries and dependencies
#


from   datetime              import datetime

print('Loading Libraries',datetime.now());

# intentionally ignoring certain waring messages that we know are not relevant to this program, like deprecated functions or changed default parameters.
# this is done to avoid cluttering the output with warnings that are not relevant to this program.

import warnings
warnings.filterwarnings("ignore")

import pandas                    as pd;
import hvplot.pandas;
from   sklearn.cluster       import KMeans;
from   sklearn.decomposition import PCA;
from   sklearn.preprocessing import StandardScaler;

print('Libraries Loaded ',datetime.now());

# displaying time for loading libraries, just to evaluate the time it takes to load libraries
# program is being tested under two architectures:
# one is a MacBook Pro M2 Ultra with 32GB of RAM.
# the other is a Alienware M18 with 32 GB of RAM.

Loading Libraries 2023-08-27 15:56:53.310052


Libraries Loaded  2023-08-27 15:56:54.739594


In [2]:
#
# Data Dicitionary, with variables used in the program
#
# dfSourceData        :  DataFrame with the source data, CryptoCurrency data
# cryptoScaledArray   :  Array with the scaled data
# cryptoTransformed   :  DataFrame with the transformed data
# coinsNames          :  Array with the coins names
# inertia1            :  Array with the inertia data
# inertia2            :  Array with the inertia data
# elbowData1          :  dictionary with the Elbow data 1
# elbowData2          :  dictionary with the Elbow data 2
# elbowDF1            :  DataFrame with the Elbow data 1
# elbowDF2            :  DataFrame with the Elbow data 2
# clustersPredicted   :  Array with the predicted clusters
# clusterPCAData      :  DataFrame with the PCA data
# clusterPCADF        :  DataFrame with the PCA data and the predicted clusters

In [3]:
# 
# Load the data into a Pandas DataFrame
#

print('Loading Data',datetime.now())
dfSourceData = pd.read_csv("Resources/crypto_market_data.csv",index_col="coin_id")

#
# Display sample data
#

print(dfSourceData.head(10))
print('Data Loaded ',datetime.now())

Loading Data 2023-08-27 15:56:54.747366
                 price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                    
bitcoin                              1.08388                     7.60278   
ethereum                             0.22392                    10.38134   
tether                              -0.21173                     0.04935   
ripple                              -0.37819                    -0.60926   
bitcoin-cash                         2.90585                    17.09717   
binancecoin                          2.10423                    12.85511   
chainlink                           -0.23935                    20.69459   
cardano                              0.00322                    13.99302   
litecoin                            -0.06341                     6.60221   
bitcoin-cash-sv                      0.92530                     3.29641   

                 price_change_percentage_14d  p

In [4]:
# Generate summary statistics
print('Summary Statistics',datetime.now())
dfSourceData.describe()

Summary Statistics 2023-08-27 15:56:54.754373


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [5]:
#
# Plot your data to see what's in your DataFrame
#
print('Plotting Data',datetime.now())
dfSourceData.hvplot.line(width=1200,height=600,rot=90)

Plotting Data 2023-08-27 15:56:54.767918


:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

### Prepare the Data

In [6]:
#
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
# According to the graph above, at least one cryptocurrency is in the thousands, while the rest are in the hundreds.
# This means that the data is skewed and needs to be normalized.
# 
print('Normalizing Data',datetime.now())
cryptoScaledArray = StandardScaler().fit_transform(dfSourceData)
print(cryptoScaledArray)
print('Data Normalized',datetime.now())

Normalizing Data 2023-08-27 15:56:54.891026
[[ 5.08529366e-01  4.93193071e-01  7.72200433e-01  2.35459633e-01
  -6.74950963e-02 -3.55953481e-01 -2.51636882e-01]
 [ 1.85445894e-01  9.34445040e-01  5.58692121e-01 -5.43409317e-02
  -2.73482725e-01 -1.15759474e-01 -1.99352110e-01]
 [ 2.17739616e-02 -7.06336853e-01 -2.16804207e-02 -6.10301536e-02
   8.00452481e-03 -5.50246924e-01 -2.82060506e-01]
 [-4.07643829e-02 -8.10928066e-01  2.49457974e-01 -5.03879651e-02
  -3.73164019e-01 -4.58258816e-01 -2.95546142e-01]
 [ 1.19303608e+00  2.00095907e+00  1.76061001e+00  5.45842065e-01
  -2.91202870e-01 -4.99847761e-01 -2.70316950e-01]
 [ 8.91870708e-01  1.32729453e+00  8.00214184e-01 -5.71478992e-02
   7.78653106e-01 -1.88231917e-01 -2.25532605e-01]
 [ 1.13972400e-02  2.57225091e+00  1.10164693e+00 -4.90495415e-01
  -9.31954023e-01  3.87758986e-01 -1.82843995e-02]
 [ 1.02529802e-01  1.50800146e+00  6.48885061e-01  3.28959245e-01
  -4.86348899e-01  6.50796233e-02 -1.55428416e-01]
 [ 7.74971820e-02  3

In [7]:
#
# Create a DataFrame with the scaled data
#
print('Before DataFrame transformation',datetime.now())
cryptoTransformed = pd.DataFrame(cryptoScaledArray, columns=['price_change_percentage_24h', 
                                                             'price_change_percentage_7d',
                                                             'price_change_percentage_14d', 
                                                             'price_change_percentage_30d', 
                                                             'price_change_percentage_60d', 
                                                             'price_change_percentage_200d',	
                                                             'price_change_percentage_1y'])
print(cryptoTransformed)
#
# Copy the crypto names from the original data
#
coinsNames                   = list(dfSourceData.index) #create a list of the coins names
print(coinsNames)
#
# Set the coinid column as index
#
cryptoTransformed['coin_id'] = coinsNames                             #create a new column with the coin names
cryptoTransformed            = cryptoTransformed.set_index('coin_id') #set the coin names as index
#
# Display sample data
#
print('After DataFrame transformation',datetime.now())
print(cryptoTransformed)


Before DataFrame transformation 2023-08-27 15:56:54.897323
    price_change_percentage_24h  price_change_percentage_7d  \
0                      0.508529                    0.493193   
1                      0.185446                    0.934445   
2                      0.021774                   -0.706337   
3                     -0.040764                   -0.810928   
4                      1.193036                    2.000959   
5                      0.891871                    1.327295   
6                      0.011397                    2.572251   
7                      0.102530                    1.508001   
8                      0.077497                    0.334297   
9                      0.448952                   -0.190684   
10                     0.331280                   -1.614844   
11                     0.034352                   -0.733026   
12                     0.155710                   -0.922491   
13                     0.262723                    1.792602

### Find the Best Value for k Using the Original Data.

In [8]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))
print('Values of k',k)

Values of k [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [9]:
#
# Create an empty list to store the inertia values
#
inertia1 = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1,n_init=10,max_iter=1000) #n_init=10,max_iter=1000 provided to avoid depcrecation warning
    k_model.fit(cryptoTransformed)
    inertia1.append(k_model.inertia_)
print('Values of Inertia ',inertia1)

Values of Inertia  [287.0, 195.82021818036043, 123.19048183836958, 79.02243535120977, 63.85866780584266, 53.05778846567061, 44.4067905846164, 37.07823336746088, 32.83218742836354, 28.165433305979256, 24.964637877542405]


In [10]:
# Create a dictionary with the data to plot the Elbow curve
print('Creating Elbow Curve',datetime.now())
elbowData1 = {"k": k, "inertia": inertia1}


# Create a DataFrame with the data to plot the Elbow curve
elbowDF1 = pd.DataFrame(elbowData1)
elbowDF1

Creating Elbow Curve 2023-08-27 15:56:55.010054


,k,inertia
0,1,287.000000
1,2,195.820218
2,3,123.190482
3,4,79.022435
4,5,63.858668
5,6,53.057788
6,7,44.406791
7,8,37.078233
8,9,32.832187
9,10,28.165433


In [11]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
print('Plotting Elbow Curve',datetime.now())
elbowDF1.hvplot.line(
    x="k",
    y="inertia",
    title= "Elbow Curve",
    xticks=k
)

Plotting Elbow Curve 2023-08-27 15:56:55.015826


:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for 'k' is **4**

### Cluster Cryptocurrencies with K-means Using the Original Data

In [12]:
# Initialize the K-Means model using the best value for k
print('Initializing K-Means Model',datetime.now())
model = KMeans(n_clusters=4, random_state=1)
print('K-Means Model Initialized ',datetime.now())

Initializing K-Means Model 2023-08-27 15:56:55.059415
K-Means Model Initialized  2023-08-27 15:56:55.059465


In [13]:
# Fit the K-Means model using the scaled data
print('Fitting K-Means Model',datetime.now())
model.fit(cryptoTransformed)
print('K-Means Model Fitted ',datetime.now())

Fitting K-Means Model 2023-08-27 15:56:55.062717
K-Means Model Fitted  2023-08-27 15:56:55.070353


In [14]:
# Predict the clusters to group the cryptocurrencies using the scaled data
k4 = model.predict(cryptoTransformed)

# Print the resulting array of cluster values.
k4


array([2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2,
       0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0],
      dtype=int32)

In [15]:
# Create a copy of the DataFrame
clustersPredicted = cryptoTransformed.copy()

In [16]:
# Add a new column to the DataFrame with the predicted clusters
clustersPredicted['predicted_cluster'] = k4

# Display sample data
clustersPredicted.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,predicted_cluster
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [17]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
clustersPredicted.hvplot.scatter(x='price_change_percentage_24h',y='price_change_percentage_7d',by='predicted_cluster',hover_cols = 'coin_id'
)

:NdOverlay   [predicted_cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

### Optimize Clusters with Principal Component Analysis.

In [18]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)


In [19]:
#
# Use the PCA model with `fit_transform` to reduce to three principal components.
#
clustersPCA = pca.fit_transform(clustersPredicted)
#
# View the first five rows of the DataFrame. 
#
clustersPCA[:5]

array([[ 0.44890795, -1.24537573, -0.8506404 ],
       [ 0.49536709, -0.899823  , -1.31755943],
       [-0.81884571,  0.07189909,  0.69501507],
       [-0.84035711,  0.08005407,  0.54435952],
       [ 0.81324004, -2.66952035, -1.64332113]])

In [20]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.34871677, 0.31363391, 0.22627118])

In [21]:
#calculate the Total Explained Variance by summing all 3 Explained Variance Ratios
sum(pca.explained_variance_ratio_)

0.8886218549859446

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 0.34871677 + 0.31363391 + 0.22627118 = 0.88862186

**Answer may change depending on re-execution of the whole code**

In [22]:
# Create a new DataFrame with the PCA data.
clusterPCAdf = pd.DataFrame(
    clustersPCA,
    columns = ["PCA1", "PCA2", "PCA3"]
)
clusterPCAdf

# Copy the crypto names from the original data
clusterPCAdf['coin_id'] = list(clustersPredicted.index)
clusterPCAdf

# Set the coinid column as index
clusterPCAdf = clusterPCAdf.set_index('coin_id')

# Display sample data
clusterPCAdf


,PCA1,PCA2,PCA3
coin_id,,,
bitcoin,0.448908,-1.245376,-0.850640
ethereum,0.495367,-0.899823,-1.317559
tether,-0.818846,0.071899,0.695015
ripple,-0.840357,0.080054,0.544360
bitcoin-cash,0.813240,-2.669520,-1.643321
binancecoin,0.822871,-1.682096,-0.905203
chainlink,0.913868,-1.128684,-2.730038
cardano,0.811102,-1.113378,-1.598655
litecoin,0.258728,-0.946884,-1.103560


### Find the Best Value for k Using the PCA Data

In [23]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,12))
k

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [24]:
# Create an empty list to store the inertia values
inertia2 = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i, random_state=1,n_init=10,max_iter=1000)
    k_model.fit(clusterPCAdf)
    inertia2.append(k_model.inertia_)
print(inertia2)

[290.6226939879639, 203.5797257883004, 112.854846058156, 44.13060225321705, 33.217427255152344, 23.37167668829717, 17.17220012359737, 13.593410892098134, 11.155098209251477, 8.958913480156944, 7.345260405169598]


In [25]:
# Create a dictionary with the data to plot the Elbow curve
elbowData2 = {"k": k, "inertia": inertia2}
# Create a DataFrame with the data to plot the Elbow curve
elbowDF2 = pd.DataFrame(elbowData2)
elbowDF2

,k,inertia
0,1,290.622694
1,2,203.579726
2,3,112.854846
3,4,44.130602
4,5,33.217427
5,6,23.371677
6,7,17.172200
7,8,13.593411
8,9,11.155098
9,10,8.958913


In [26]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbowDF2.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best k-value is  `k=4` when using PCA data


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, it is the same `k` value as found using the original data

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [27]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4, random_state=1)

In [28]:
# Fit the K-Means model using the PCA data
model.fit(clusterPCAdf)

KMeans(n_clusters=4, random_state=1)

In [29]:
# Predict the clusters to group the cryptocurrencies using the PCA data
k4 = model.predict(clusterPCAdf)
# Print the resulting array of cluster values.
k4

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3, 1, 0, 0, 2, 0, 0, 0, 0],
      dtype=int32)

In [30]:
# Create a copy of the DataFrame with the PCA data
copyClusterPCAdf= clusterPCAdf.copy()

# Add a new column to the DataFrame with the predicted clusters
copyClusterPCAdf['predicted_cluster'] = k4

# Display sample data
copyClusterPCAdf

,PCA1,PCA2,PCA3,predicted_cluster
coin_id,,,,
bitcoin,0.448908,-1.245376,-0.850640,1
ethereum,0.495367,-0.899823,-1.317559,1
tether,-0.818846,0.071899,0.695015,0
ripple,-0.840357,0.080054,0.544360,0
bitcoin-cash,0.813240,-2.669520,-1.643321,1
binancecoin,0.822871,-1.682096,-0.905203,1
chainlink,0.913868,-1.128684,-2.730038,1
cardano,0.811102,-1.113378,-1.598655,1
litecoin,0.258728,-0.946884,-1.103560,1


In [31]:
# Create a scatter plot using hvPlot by setting 
# `x="PCA1"` and `y="PCA2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
copyClusterPCAdf.hvplot.scatter(
    x="PCA1",
    y="PCA2",
    by = 'predicted_cluster',
    hover_cols='coin_id'
)

:NdOverlay   [predicted_cluster]
   :Scatter   [PCA1]   (PCA2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [32]:
# Composite plot to contrast the Elbow curves
elbowDF1.hvplot.line(x="k", y="inertia", title="Elbow Curve 1", xticks=k) + \
elbowDF2.hvplot.line(x="k", y="inertia", title="Elbow Curve 2", xticks=k)

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [33]:
# Composite plot to contrast the clusters
clustersPredicted.hvplot.scatter( x='price_change_percentage_24h',  
                                  y='price_change_percentage_7d', 
                                  by='predicted_cluster', hover_cols = 'coin_id') + \
copyClusterPCAdf.hvplot.scatter(x="PCA1", y="PCA2", by = 'predicted_cluster', hover_cols='coin_id')

:Layout
   .NdOverlay.I  :NdOverlay   [predicted_cluster]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [predicted_cluster]
      :Scatter   [PCA1]   (PCA2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** The impact of using PCA data resulted in tighter grouped clusters, with more entries within cluster 0 and cluster 1 than the original analysis did.